In [1]:
# Imports & Inits

from datetime import datetime
from datetime import timezone
import polars as pl
import os

from utils import two_point_calibration, average_bottle, plot_sensor_measurement


#------------
sensor_id = 4
#------------

data_directory = "/Users/patrickaigner/Documents/PROJECTS/ACROPOLIS/Software/acropolis-visualisation/data"
df = pl.scan_parquet(os.path.join(data_directory, "old_db_renamed_measurements.parquet"))

df = df.filter(pl.col("system_name") == f"tum-esm-midcost-raspi-{sensor_id}") \
    .sort(pl.col("creation_timestamp")) \
    .filter(pl.col("gmp343_filtered") > 0.0) \
    .select(pl.col("creation_timestamp"),
                pl.col("gmp343_filtered"),
                pl.col("sht45_humidity"))

FileNotFoundError: No such file or directory: ...ECTS/ACROPOLIS/Software/acropolis-visualisation/data/old_db_renamed_measurements.parquet

In [ ]:
#600
start_date = datetime(2023, 9, 18, 12, 39, 30).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 9, 18, 13, 8, 0).replace(tzinfo=timezone.utc)

df_p_600 = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

plot_sensor_measurement(df_p_600.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_p_600.collect(), col_name="sht45_humidity")

#400
start_date = datetime(2023, 9, 18, 13, 10, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 9, 18, 13, 38, 0).replace(tzinfo=timezone.utc)

df_p_400 = df.sort("creation_timestamp").filter(pl.col("creation_timestamp").is_between(start_date, end_date))

plot_sensor_measurement(df_p_400.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_p_400.collect(), col_name="sht45_humidity")


In [ ]:
measured_values = [None, None]
true_values = [427.38, 610.95]

# 400 ppm

data = df_p_400.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
measured_values[0] = average_bottle(data)

# 600 ppm
data = df_p_600.select(pl.col("gmp343_filtered")).collect().to_series().to_list()

measured_values[1] = average_bottle(data)

# calculate slope and intercept
slope, intercept = two_point_calibration(measured_values, true_values)
print(f"acropolis_slope = {slope}, acropolis_intercept = {intercept}")

Uncut average: 423.34550561797766
Cut average: 417.1120689655173
Uncut average: 597.4382857142856
Cut average: 597.4605263157896
acropolis_slope = 1.0178628788793624, acropolis_intercept = 2.8171086674313415
